In [1]:
import classes
import crossover
import mutation
import fitness
import pandas as pd
import numpy as np
import random

from deap import base, creator, tools

In [2]:
# Get data from csv file
def get_data(csv_file):
  data_df = pd.read_csv(csv_file, dtype={
    'pos_name': str,
    'latitude': float,
    'longitude': float,
    'address': str,
    'entree_time': int,
    'unloading_time': int,
    'journey2pos_time': int,
    'delivery_time': int,
    'journey2unloadingpoint_time': int,
    'checkout_time': int,
    'min_travels': int,
    'max_travels': int,
    'extra_times': int
  })
  
  data = data_df.to_records(index=False).tolist()
  
  return data

In [3]:
# shuffle the Point_of_Sale list of an individual
def shuffle_sequence(individual):
  random.shuffle(individual)
  return individual

In [4]:
# Read CSV and create a initial list of Point_of_Sale
readed_points_of_sale = get_data('data.csv')

# Create a list of Point_of_Sale
pos_list = [
  classes.Point_of_Sale(
    point[0], point[1], point[2], point[3],
    point[4], point[5], point[6], point[7],
    point[8], point[9], point[10], point[11], point[12]
  )
  for point in readed_points_of_sale
]

In [5]:
distance_df = mutation.build_distance_dataframe(pos_list)
distance_df

,Zona4,Avia,Majadas,Miraflores,Oakland,Cayalá,NaranjoMall,ZonaPortales
Zona4,0.000000,2.117936,5.118645,4.329277,2.565118,3.307246,4.556509,4.489057
Avia,2.117936,0.000000,5.698270,4.843962,0.695130,3.222121,6.326063,6.032898
Majadas,5.118645,5.698270,0.000000,0.854504,6.389157,8.344023,3.755488,8.944657
Miraflores,4.329277,4.843962,0.854504,0.000000,5.535174,7.518521,3.704805,8.314381
Oakland,2.565118,0.695130,6.389157,5.535174,0.000000,2.824902,6.941083,6.041197
Cayalá,3.307246,3.222121,8.344023,7.518521,2.824902,0.000000,7.622287,3.990709
NaranjoMall,4.556509,6.326063,3.755488,3.704805,6.941083,7.622287,0.000000,6.499582
ZonaPortales,4.489057,6.032898,8.944657,8.314381,6.041197,3.990709,6.499582,0.000000


In [6]:
# Using DEAP library
# weights is a tuple of -1.0s which means we want to Minimize two values: Distance and Time
creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

# initializing DEAP's toolbox
toolbox = base.Toolbox()
# individual's attribute sequence will shuffle a list of POS at creation
toolbox.register("sequence", shuffle_sequence, pos_list)
# individual inherits from Individual and will have the attribute 'sequence'
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.sequence)
# population will be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# mate function from ./crossover.py
toolbox.register("mate", crossover.cxOrdered)
# mutation function that uses DEAP's function mutShuffleIndexes. 
# indpb is the probability of each attribute to be exchanged to another position
toolbox.register("mutate", mutation.smart_mutation_with_df, distance_df=distance_df)
# toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)
# tournsize = The number of individuals participating in each tournament
toolbox.register("select", tools.selTournament, tournsize=2)
# fitness function from ./crossover.py
toolbox.register("evaluate", fitness.evaluate_osrm)

In [7]:
# Main Genetic Algorithm
def main(matepb, mutpb, ngen, npop):
  """Executes the main genetic algorithm.
  :param matepb: Probability that two individuals cross.
  :param mutpb: Probability that an individual mutates.
  :param ngen: Number of generations the population will have
  :param npop: How many individuals will make up the population
  :returns: a :term:`list` containing the las generation of a population
  """

  population = toolbox.population(n=npop)
  # Evaluate the entire population
  fitnesses = map(toolbox.evaluate, population)

  for individual, fitness in zip(population, fitnesses):
    individual.fitness.values = fitness

  for generation in range(ngen):
    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))

    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if random.random() < matepb:
        toolbox.mate(child1, child2)
        # Invalidates the fitness of the offspring after crossing
        del child1.fitness.values
        del child2.fitness.values

    for mutant in offspring:
      if random.random() < mutpb:
        toolbox.mutate(mutant)
        # Invalidates the fitness of the mutant after mutation
        del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
      ind.fitness.values = fit

    elite = tools.selBest(population, 1)  # Keep the best individual

    # The population is entirely replaced by the offspring
    population[:] = offspring
    population[:1] = elite
  
  return population  

In [12]:
# GA Execution
# (YYYY-MM-DD)T(HH:mm:ss.ms)(UTC-6)
crossover.departure_time = "2024-09-27T09:00:00.000000-06:00"

pop = main(matepb=0.5, mutpb=0.5, ngen=100, npop=100)

In [14]:
# Show results
def extract_names(individual):
    return [pos.name for pos in individual]

pop_data = np.array([extract_names(ind) for ind in pop])
pop_df = pd.DataFrame(pop_data)

fitnesses_list = []
for ind in pop:
  fitnesses_list.append(ind.fitness.values)

fitness_df = pd.DataFrame(fitnesses_list, columns=['Distance', 'Time'])

result_df = pd.concat([pop_df, fitness_df], axis=1)
result_df.value_counts().to_frame(name="f")

f
0            1       2       3            4            5           6           7           Distance Time      
Oakland      Avia    Zona4   Cayalá       ZonaPortales NaranjoMall Majadas     Miraflores  41387.6  3849.2  33
                                                                   Miraflores  Majadas     42358.8  3914.8  14
Avia         Oakland Zona4   Cayalá       ZonaPortales NaranjoMall Majadas     Miraflores  43856.5  4051.6   9
Oakland      Avia    Zona4   ZonaPortales Cayalá       NaranjoMall Majadas     Miraflores  47079.9  4297.2   8
             Zona4   Avia    Cayalá       ZonaPortales NaranjoMall Majadas     Miraflores  44432.3  4062.3   6
Cayalá       Avia    Zona4   Oakland      ZonaPortales NaranjoMall Majadas     Miraflores  58361.7  5154.4   5
Oakland      Avia    Zona4   ZonaPortales Cayalá       NaranjoMall Miraflores  Majadas     48051.1  4362.8   4
                             Cayalá       ZonaPortales Miraflores  Majadas     NaranjoMall 50745.4  4373.0   4
                                                                   NaranjoMall Majadas     50780.6  4356.7   3
Avia         Oakland Zona4   Cayalá       ZonaPortales NaranjoMall Miraflores  Majadas     44827.7  4117.2   3
                             ZonaPortales Cayalá       NaranjoMall Majadas     Miraflores  49548.8  4499.6   3
             Cayalá  Zona4   Oakland      ZonaPortales NaranjoMall Majadas     Miraflores  55252.5  4942.5   2
             Oakland Zona4   Cayalá       ZonaPortales Miraflores  Majadas     NaranjoMall 53214.3  4575.4   1
             Zona4   Oakland Cayalá       ZonaPortales NaranjoMall Majadas     Miraflores  47473.3  4286.7   1
                                                       Miraflores  Majadas     NaranjoMall 56831.1  4810.5   1
Oakland      Zona4   Avia    Cayalá       ZonaPortales NaranjoMall Miraflores  Majadas     45403.5  4127.9   1
                             ZonaPortales Cayalá       NaranjoMall Majadas     Miraflores  51451.0  4694.7   1
ZonaPortales Avia    Zona4   Oakland      Cayalá       NaranjoMall Majadas     Miraflores  63135.9  5620.3   1

In [10]:
routes_df = pd.DataFrame(fitness.google_res_dict).transpose()
routes_df.to_clipboard()
routes_df

""


In [11]:
routes_df = pd.DataFrame(fitness.osmr_res_dict).transpose()
routes_df.to_clipboard()
routes_df

,distance,duration
"NaranjoMall,Miraflores,Oakland,Cayalá,Majadas,ZonaPortales,Zona4,Avia",71214.3,6035.0
"Avia,Cayalá,Majadas,Zona4,NaranjoMall,Miraflores,ZonaPortales,Oakland",69176.6,6017.2
"Miraflores,Majadas,Cayalá,Oakland,ZonaPortales,Avia,NaranjoMall,Zona4",73223.5,6304.4
"NaranjoMall,Oakland,Miraflores,ZonaPortales,Avia,Majadas,Zona4,Cayalá",81996.9,6857.6
"Cayalá,Oakland,Avia,Majadas,Miraflores,ZonaPortales,NaranjoMall,Zona4",62020.6,5383.3
...,...,...
"Avia,Cayalá,Zona4,Oakland,ZonaPortales,NaranjoMall,Miraflores,Majadas",56223.7,5008.1
"Avia,ZonaPortales,Zona4,Cayalá,NaranjoMall,Majadas,Miraflores,Oakland",56312.4,5156.9
"Cayalá,Zona4,Majadas,Miraflores,Oakland,ZonaPortales,NaranjoMall,Avia",64112.5,5622.6
"Oakland,Zona4,Cayalá,Avia,ZonaPortales,Miraflores,NaranjoMall,Majadas",61017.7,5220.1
